In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split


# Punto 1

In [9]:
respondieron = pd.read_csv('respondieron.csv')

In [10]:
respondieron.shape

(14647, 2)

In [11]:
norespondieron = pd.read_csv('norespondieron.csv')

In [12]:
respondieronconaños = pd.read_csv('respondieron_con_años.csv')

In [13]:
respondieronconaños

,cond. de actividad,PEA,ano4
0,3.0,2.0,2004.0
1,1.0,1.0,2004.0
2,3.0,2.0,2004.0
3,1.0,1.0,2004.0
4,3.0,2.0,2004.0
...,...,...,...
14642,3.0,2.0,2024.0
14643,2.0,1.0,2024.0
14644,1.0,1.0,2024.0
14645,1.0,1.0,2024.0


In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [15]:
respondieron["desocupado"] = (respondieronconaños["PEA"] == 1).astype(int)

In [16]:
print(respondieronconaños["ano4"].unique()) #estan bien cargados los dos años (2004) y (2024)

[2004. 2024.]


In [35]:
respondieronconaños["desocupado"] = (respondieronconaños["PEA"] == 1).astype(int) 
#Para cada valor 1 de PEA, la tabla de desocupado devuelve 1, para otro valor devuelve 0. Es un booleano.

In [37]:
for año in respondieronconaños["ano4"].unique():
    print(f"\nProcesando año: {año}")
    
    df_anual = respondieronconaños[respondieronconaños["ano4"] == año]
    
    y = df_anual["desocupado"] #Esta será nuestra variable independiente
    
    X = pd.get_dummies(df_anual["PEA"], prefix="PEA", drop_first=True) #Usamos solo PEA, con valores booleanos como predictor
    X = X.astype(int) #Consideramos más comodo trabajar con unos y ceros
    X = X.copy() #Esto para evitar warnings y errores
    X["intercept"] = 1 #La columna de unos es útil para la ordenada al origen, Beta sub 0
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=444) #Requerimientos de la consigna
    print(f"Año {año}:")
    print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")
    #Podemos ver ahora la cantidad de filas y columnas que tienen nuestras muestras en cada año.


Procesando año: 2004.0
Año 2004.0:
Train shape: (5352, 2), Test shape: (2295, 2)

Procesando año: 2024.0
Año 2024.0:
Train shape: (4900, 2), Test shape: (2100, 2)


## Evitamos Multicolinealidad
Decidimos eliminar una columna PEA de las dos que generan las dummies, porque ya sabemos que si una es verdadera, la otra es falsa, entonces podemos evitarnos esta relación excluyente que tienen. Tampoco es útil la multicolinealidad para interpretar los coeficientes, por lo que para evitar confusiones es mejor no tenerlo en nuestro modelo. Así que solo tendremos en cuenta PEA 2, que adopta 1 si es Población inactiva, y adopta 0 si es Población economicamente activa (donde se encuentran los desocupados)
Por otro lado, por si no se notó, decidimos eliminar Cond. de actividad, debido a que lo consideramos algo ilógico desde lo predictivo, porque estariamos prediciendo nuestra variable objetivo con una columna que incluye directamente la variable objetivo (Multicolinealidad muy alta)

In [40]:
X

,PEA_2.0,intercept
7647,0,1
7648,1,1
7649,1,1
7650,0,1
7651,0,1
...,...,...
14642,1,1
14643,0,1
14644,0,1
14645,0,1


In [42]:
common_cols = X_train.columns.intersection(X_test.columns)


In [44]:
mean_train = X_train[common_cols].mean()
mean_test = X_test[common_cols].mean()

In [46]:
diff_table = pd.DataFrame({
    "Media Entrenamiento": mean_train,
    "Media Testeo": mean_test,
    "Diferencia Absoluta": (mean_train - mean_test).abs()
})
diff_table = diff_table.sort_values("Diferencia Absoluta", ascending=False)
print(diff_table)

           Media Entrenamiento  Media Testeo  Diferencia Absoluta
PEA_2.0               0.487347      0.517143             0.029796
intercept             1.000000      1.000000             0.000000


### Conclusión
La tabla de diferencia de medias muestra que las medias de las variables explicativas en los conjuntos de entrenamiento y testeo son muy similares. La variable PEA presenta una diferencia absoluta pequeña de aproximadamente 0.03, lo que sugiere una distribución equilibrada.

# Punto 2

In [100]:
edad = pd.read_csv("edad.csv")

In [102]:
edad2 = pd.read_csv("edad2.csv")

In [104]:
horastrab = pd.read_csv("horastrab.csv")

In [106]:
salario_semanal = pd.read_csv("salario semanal.csv")

In [108]:
educ = pd.read_csv("educ.csv")

In [110]:
Ocupados=pd.read_csv("Ocupados.csv")

In [112]:
OcupadosEPH = Ocupados.combine_first(edad).dropna()


In [114]:
OcupadosEPH=OcupadosEPH.combine_first(edad2).dropna()

In [116]:
OcupadosEPH = OcupadosEPH.combine_first(educ).dropna()

In [118]:
OcupadosEPH = OcupadosEPH.combine_first(horastrab).dropna()

In [120]:
OcupadosEPH = OcupadosEPH.combine_first(salario_semanal).dropna()
OcupadosEPH

,año,edad,edad2,educ,estado,horastrab,salario semanal
0,2004.0,79.0,6241.0,10.0,1.0,0.0,0.0
1,2004.0,67.0,4489.0,16.0,1.0,0.0,0.0
2,2004.0,74.0,5476.0,14.0,1.0,0.0,0.0
3,2004.0,54.0,2916.0,10.0,1.0,59.0,60000.0
4,2004.0,51.0,2601.0,9.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
7195,2024.0,54.0,2916.0,7.0,1.0,40.0,0.0
7196,2024.0,24.0,576.0,9.0,1.0,0.0,0.0
7197,2024.0,20.0,400.0,10.0,1.0,0.0,0.0
7198,2024.0,1.0,1.0,7.0,1.0,0.0,0.0


In [70]:
import statsmodels.api as sm

In [72]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


df_train = X_train.copy()
df_train["salario_semanal"] = y_train


In [74]:
for año in salario_semanal["año"].unique():
    print(f"\nProcesando año: {año}")
    
    y = salario_semanal[["salario semanal"]] #Esta será nuestra variable independiente
    
    X = edad["edad"] #Predictor/es
    X = X.astype(int) 
    X = X.copy() #Esto para evitar warnings y errores
    X["intercept"] = 1 #La columna de unos es útil para la ordenada al origen, Beta sub 0
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=444) #Requerimientos de la consigna
    print(f"Año {año}:")
    print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")
    #Podemos ver ahora la cantidad de filas y columnas que tienen nuestras muestras en cada año.


Procesando año: 2004.0


ValueError: Found input variables with inconsistent numbers of samples: [14699, 14698]